# Forecasting Showdown: Prediction Markets vs. Frontier LLMs

## Project Overview

This project compares forecasting accuracy across three categories of forecasters:

1. **Prediction Markets** (Polymarket + Kalshi) â€” real-money markets where participants trade on event outcomes
2. **Frontier LLMs â€” Vanilla** (GPT-5, Gemini, Claude) â€” prompted with only the question, relying on training data
3. **Frontier LLMs â€” Tool-Augmented** (same models with real-time data tools) â€” given access to FRED and EIA APIs

### Metrics

**Brier Score** measures calibration â€” how well probability estimates match actual outcomes:

$$BS = \frac{1}{N} \sum_{i=1}^{N} (p_i - o_i)^2$$

- $p_i$ = predicted probability, $o_i$ = outcome (0 or 1)
- Lower is better: 0 = perfect, 0.25 = no skill (always predicting 50%), 1 = worst

**Hypothetical Returns** test profitability via a threshold-based betting strategy against prediction market prices.

### Domains
- **Federal Funds Rate**: Will the Fed cut rates at upcoming FOMC meetings?
- **Gas Prices**: Will US national average gasoline prices exceed/fall below specific thresholds?

In [ ]:
# Environment Setup
# Loads API keys from a .env file (local) or Colab Secrets (Google Colab).
import os
import sys
from pathlib import Path

# --- Google Colab Support ---
if "google.colab" in sys.modules:
    print("Running in Google Colab")
    print("Add your API keys to Colab Secrets (key icon in the left sidebar).")
    try:
        from google.colab import userdata
        for key in [
            "OPENAI_API_KEY", "GOOGLE_API_KEY", "GEMINI_API_KEY",
            "ANTHROPIC_API_KEY", "FRED_API_KEY", "EIA_API_KEY",
        ]:
            try:
                os.environ[key] = userdata.get(key)
            except Exception:
                pass  # Key not set in Colab secrets -- that's ok
    except ImportError:
        pass

# --- Local: load from .env file ---
else:
    try:
        from dotenv import load_dotenv
        env_file = Path(".env")
        if env_file.exists():
            load_dotenv(env_file, override=True)
            print("Loaded API keys from .env")
        else:
            print("No .env file found -- using existing environment variables.")
            print("Copy .env.example to .env and fill in your keys.")
    except ImportError:
        print("python-dotenv not installed -- run: pip install python-dotenv")

# --- Report key status ---
KEYS = {
    "OPENAI_API_KEY":    "Required -- GPT models",
    "GOOGLE_API_KEY":    "Required -- Gemini models",
    "GEMINI_API_KEY":    "Required -- Gemini models (alias)",
    "ANTHROPIC_API_KEY": "Required -- Claude models",
    "FRED_API_KEY":      "Required -- Fed rate data  (free at fred.stlouisfed.org/docs/api/api_key.html)",
    "EIA_API_KEY":       "Required -- Gas price data (free at eia.gov/opendata)",
}
all_set = True
for key, desc in KEYS.items():
    status = "set" if os.environ.get(key) else "MISSING"
    if not os.environ.get(key):
        all_set = False
    print(f"  {status}  {key}  ({desc})")

if not all_set:
    print("
Some keys are missing. See README.md for setup instructions.")
else:
    print("
All API keys loaded.")

In [2]:
import hashlib
import json
import re
import time
from datetime import datetime, timedelta
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from fredapi import Fred
from sklearn.metrics import brier_score_loss
from tqdm.auto import tqdm

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool

# --- Model Configuration ---
MODELS = {
    "gemini": {
        "name": "Gemini 2.5 Flash Lite",
        "provider": "google",
        "model_id": "gemini-2.5-flash-lite",
    },
    "gpt": {
        "name": "GPT-5",
        "provider": "openai",
        "model_id": "gpt-5-2025-08-07",
    },
    "claude": {
        "name": "Claude Sonnet 4.5",
        "provider": "anthropic",
        "model_id": "claude-sonnet-4-5-20250929",
    },
}

# --- API Keys ---
FRED_API_KEY = os.environ.get("FRED_API_KEY", "")
EIA_API_KEY = os.environ.get("EIA_API_KEY", "")

# --- Prediction Market API Base URLs ---
POLYMARKET_GAMMA_BASE = "https://gamma-api.polymarket.com"
POLYMARKET_CLOB_BASE = "https://clob.polymarket.com"
KALSHI_BASE = "https://api.elections.kalshi.com/trade-api/v2"

# --- FRED Series IDs ---
FRED_SERIES = {
    "fed_funds_rate": "DFF",
    "fed_funds_target_upper": "DFEDTARU",
    "fed_funds_target_lower": "DFEDTARL",
}

# --- EIA Endpoint ---
EIA_GAS_PRICE_URL = "https://api.eia.gov/v2/petroleum/pri/gp/data/"

# --- 2026 FOMC Meeting Dates (announcement day) ---
FOMC_DATES_2026 = [
    {"meeting": "January",   "date": "2026-01-28"},
    {"meeting": "March",     "date": "2026-03-18"},
    {"meeting": "April",     "date": "2026-04-29"},
    {"meeting": "June",      "date": "2026-06-17"},
    {"meeting": "July",      "date": "2026-07-29"},
    {"meeting": "September", "date": "2026-09-16"},
    {"meeting": "October",   "date": "2026-10-28"},
    {"meeting": "December",  "date": "2026-12-09"},
]

# Ensure cache directory exists
Path("cache").mkdir(exist_ok=True)

print("Configuration loaded.")
print(f"FRED API key: {'set' if FRED_API_KEY else 'MISSING'}")
print(f"EIA API key: {'set' if EIA_API_KEY else 'MISSING'}")

Configuration loaded.
FRED API key: MISSING
EIA API key: MISSING


In [ ]:
# --- Caching Infrastructure ---
# Same pattern as lab_02: JSON file-based cache to avoid redundant API calls

CACHE_FILE = Path("cache/response_cache.json")


def load_cache():
    if CACHE_FILE.exists():
        return json.loads(CACHE_FILE.read_text())
    return {}


def save_cache(cache):
    CACHE_FILE.write_text(json.dumps(cache, indent=2))


def get_cache_key(prefix, *args):
    key_str = f"{prefix}:" + ":".join(str(a) for a in args)
    return hashlib.sha256(key_str.encode()).hexdigest()[:16]


def cached_call(cache, key, func, *args, **kwargs):
    """Execute func if key not in cache; otherwise return cached result."""
    if key in cache:
        return cache[key]
    result = func(*args, **kwargs)
    cache[key] = result
    save_cache(cache)
    return result


print("Caching infrastructure ready.")

# Section 1: Data Collection

We pull data from four external sources:

| Source | What It Provides | API | Auth |
|--------|-----------------|-----|------|
| **FRED** (St. Louis Fed) | Federal funds rate (daily), target rate range | `fredapi` Python library | Free API key |
| **EIA** (Energy Information Administration) | Weekly US retail gasoline prices | REST API v2 | Free API key |
| **Polymarket** | Prediction market probabilities | Gamma + CLOB REST APIs | None |
| **Kalshi** | Prediction market probabilities | REST API v2 | None (public data) |

The FRED and EIA data serve two purposes:
1. **Ground truth** for resolving our binary questions
2. **Tool data** for the tool-augmented LLM category

In [ ]:
# --- 1.1 FRED: Federal Funds Rate ---

def fetch_fred_data():
    """Fetch federal funds rate data from FRED."""
    fred = Fred(api_key=FRED_API_KEY)

    fed_funds = fred.get_series("DFF", observation_start="2024-01-01")
    target_upper = fred.get_series("DFEDTARU", observation_start="2024-01-01")
    target_lower = fred.get_series("DFEDTARL", observation_start="2024-01-01")

    return {
        "fed_funds_rate": fed_funds,
        "target_upper": target_upper,
        "target_lower": target_lower,
    }


fred_data = fetch_fred_data()
print(f"Fed funds rate: {len(fred_data['fed_funds_rate'])} observations")
print(f"Latest effective rate: {fred_data['fed_funds_rate'].dropna().iloc[-1]:.2f}%")
print(
    f"Current target range: {fred_data['target_lower'].dropna().iloc[-1]:.2f}%"
    f" - {fred_data['target_upper'].dropna().iloc[-1]:.2f}%"
)

In [ ]:
# --- 1.2 EIA: US Retail Gasoline Prices ---

def fetch_eia_gas_prices(n_weeks=200):
    """Fetch weekly US regular gasoline retail prices from EIA API v2."""
    params = {
        "api_key": EIA_API_KEY,
        "frequency": "weekly",
        "data[0]": "value",
        "facets[product][]": "EPMR",   # Regular gasoline, all formulations
        "facets[duoarea][]": "NUS",     # National US
        "sort[0][column]": "period",
        "sort[0][direction]": "desc",
        "offset": 0,
        "length": n_weeks,
    }
    response = requests.get(EIA_GAS_PRICE_URL, params=params)
    response.raise_for_status()
    data = response.json()

    records = data["response"]["data"]
    df = pd.DataFrame(records)
    df["period"] = pd.to_datetime(df["period"])
    df["value"] = pd.to_numeric(df["value"])
    df = df.sort_values("period").reset_index(drop=True)
    return df


gas_prices = fetch_eia_gas_prices()
print(f"Gas price data: {len(gas_prices)} weekly observations")
print(
    f"Latest: ${gas_prices['value'].iloc[-1]:.3f}/gal"
    f" (week of {gas_prices['period'].iloc[-1].strftime('%Y-%m-%d')})"
)

In [ ]:
# --- 1.3 Polymarket ---

def fetch_polymarket_events(search_term, limit=50):
    """Search Polymarket Gamma API for events matching a term."""
    url = f"{POLYMARKET_GAMMA_BASE}/events"
    params = {"closed": "false", "limit": limit}
    response = requests.get(url, params=params)
    response.raise_for_status()
    events = response.json()

    relevant = []
    for event in events:
        title = event.get("title", "").lower()
        if search_term.lower() in title:
            relevant.append(event)
    return relevant


def fetch_polymarket_price_history(token_id, interval="max", fidelity=60):
    """Fetch price history for a Polymarket CLOB token."""
    url = f"{POLYMARKET_CLOB_BASE}/prices-history"
    params = {"market": token_id, "interval": interval, "fidelity": fidelity}
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()
    history = data.get("history", [])
    df = pd.DataFrame(history)
    if not df.empty:
        df["timestamp"] = pd.to_datetime(df["t"], unit="s")
        df["price"] = df["p"].astype(float)
    return df


# Search for relevant markets
poly_fed_events = fetch_polymarket_events("fed")
poly_gas_events = fetch_polymarket_events("gas")
print(f"Polymarket: {len(poly_fed_events)} Fed-related events, {len(poly_gas_events)} gas-related events")

for e in poly_fed_events[:5]:
    print(f"  Fed: {e.get('title', 'N/A')}")
for e in poly_gas_events[:5]:
    print(f"  Gas: {e.get('title', 'N/A')}")

In [ ]:
# --- 1.4 Kalshi ---

def fetch_kalshi_markets(series_ticker, status="open", limit=100):
    """Fetch markets from Kalshi for a given series."""
    url = f"{KALSHI_BASE}/markets"
    params = {"series_ticker": series_ticker, "status": status, "limit": limit}
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()
    return data.get("markets", [])


# Fetch Fed rate and gas price markets
kalshi_fed = fetch_kalshi_markets("KXFED")
kalshi_gas = fetch_kalshi_markets("KXAAAGASM")
print(f"Kalshi: {len(kalshi_fed)} Fed rate markets (KXFED), {len(kalshi_gas)} gas price markets (KXAAAGASM)")

print("\nFed rate markets:")
for m in kalshi_fed[:5]:
    yes_bid = m.get("yes_bid", m.get("last_price", "N/A"))
    print(f"  {m.get('ticker', 'N/A')}: {m.get('title', 'N/A')} | Yes: {yes_bid}")

print("\nGas price markets:")
for m in kalshi_gas[:5]:
    yes_bid = m.get("yes_bid", m.get("last_price", "N/A"))
    print(f"  {m.get('ticker', 'N/A')}: {m.get('title', 'N/A')} | Yes: {yes_bid}")

# Section 2: Binary Question Design

## Design Principles
1. **Clear resolution criteria**: each question has an unambiguous yes/no outcome
2. **Authoritative data source**: resolution determined by a specific FRED/EIA data release
3. **Alignment with prediction markets**: questions map to existing Kalshi/Polymarket contracts where possible
4. **Diverse time horizons**: mix of near-term and medium-term questions

## Question Categories

### Category A: Federal Funds Rate (FOMC Decisions)
**Template**: "Will the Fed cut the federal funds rate at the [Month] 2026 FOMC meeting?"
- Resolves YES if the FRED target rate upper bound (`DFEDTARU`) decreases after the meeting
- Resolves NO otherwise

### Category B: US Retail Gas Prices
**Template**: "Will the US national average gas price exceed $X.XX per gallon by [date]?"
- Resolves YES if EIA weekly price exceeds the threshold
- Thresholds set relative to the current price

In [ ]:
# --- 2.1 Question Generation ---

def generate_fed_questions(fomc_dates, current_rate_upper):
    """Generate binary questions about Fed rate cuts for upcoming FOMC meetings."""
    questions = []
    for meeting in fomc_dates:
        meeting_date = datetime.strptime(meeting["date"], "%Y-%m-%d")
        if meeting_date > datetime.now():
            questions.append({
                "id": f"fed_cut_{meeting['meeting'].lower()}_2026",
                "category": "fed_rate",
                "text": (
                    f"Will the Fed cut the federal funds rate at the"
                    f" {meeting['meeting']} 2026 FOMC meeting?"
                ),
                "resolution_date": meeting["date"],
                "resolution_source": "FRED DFEDTARU",
                "current_rate": float(current_rate_upper),
                "kalshi_series": "KXFED",
                "meeting_month": meeting["meeting"].lower(),
            })
    return questions


def generate_gas_questions(current_price, weeks_ahead=(4, 8, 12)):
    """Generate binary questions about gas prices at various thresholds."""
    questions = []
    offsets = [0.25, 0.50, -0.25]

    for weeks in weeks_ahead:
        target_date = datetime.now() + timedelta(weeks=weeks)
        target_date_str = target_date.strftime("%Y-%m-%d")

        for offset in offsets:
            threshold = round(current_price + offset, 2)
            direction = "exceed" if offset > 0 else "fall below"

            questions.append({
                "id": f"gas_{'above' if offset > 0 else 'below'}_{threshold:.2f}_{weeks}w",
                "category": "gas_price",
                "text": (
                    f"Will the US national average gas price {direction}"
                    f" ${threshold:.2f}/gal by the week of {target_date_str}?"
                ),
                "resolution_date": target_date_str,
                "resolution_source": "EIA Weekly Retail Gasoline Prices",
                "threshold": threshold,
                "current_price": float(current_price),
                "kalshi_series": "KXAAAGASM",
            })
    return questions


# Generate questions
current_rate = fred_data["target_upper"].dropna().iloc[-1]
current_gas = gas_prices["value"].iloc[-1]

fed_questions = generate_fed_questions(FOMC_DATES_2026, current_rate)
gas_questions = generate_gas_questions(current_gas)
all_questions = fed_questions + gas_questions

print(f"Generated {len(fed_questions)} Fed rate questions and {len(gas_questions)} gas price questions")
print(f"Total: {len(all_questions)} binary questions\n")

for q in all_questions:
    print(f"  [{q['id']}] {q['text']}")

In [ ]:
# --- 2.2 Collect Prediction Market Probabilities ---

def match_kalshi_market(markets, question):
    """Find the Kalshi market that best matches our question."""
    for m in markets:
        title_lower = m.get("title", "").lower()
        if question["category"] == "fed_rate":
            month = question["meeting_month"]
            if month in title_lower and ("cut" in title_lower or "rate" in title_lower):
                return m
        elif question["category"] == "gas_price":
            threshold_str = f"{question['threshold']:.2f}"
            if threshold_str in title_lower or str(question["threshold"]) in title_lower:
                return m
    return None


def match_polymarket_event(events, question):
    """Find the Polymarket event that best matches our question."""
    for e in events:
        title_lower = e.get("title", "").lower()
        if question["category"] == "fed_rate":
            month = question["meeting_month"]
            if month in title_lower and "fed" in title_lower:
                return e
        elif question["category"] == "gas_price":
            if "gas" in title_lower:
                return e
    return None


def get_market_price(market_obj, source):
    """Extract the YES probability from a market object."""
    if source == "kalshi":
        # Kalshi prices are in cents (0-100) or dollars (0-1)
        for key in ["yes_bid", "last_price", "yes_ask"]:
            val = market_obj.get(key)
            if val is not None:
                val = float(val)
                return val / 100 if val > 1 else val
    elif source == "polymarket":
        # Polymarket markets contain outcomes with prices
        markets = market_obj.get("markets", [])
        if markets:
            for mkt in markets:
                price = mkt.get("outcomePrices")
                if price:
                    prices = json.loads(price) if isinstance(price, str) else price
                    if prices:
                        return float(prices[0])  # YES price
        # Try top-level price
        if "price" in market_obj:
            return float(market_obj["price"])
    return np.nan


def collect_market_probabilities(questions, kalshi_fed, kalshi_gas, poly_fed, poly_gas):
    """Collect prediction market probabilities for all questions."""
    results = []

    for q in questions:
        row = {"question_id": q["id"], "question_text": q["text"]}

        # Kalshi
        kalshi_markets = kalshi_fed if q["category"] == "fed_rate" else kalshi_gas
        match = match_kalshi_market(kalshi_markets, q)
        if match:
            row["kalshi_prob"] = get_market_price(match, "kalshi")
            row["kalshi_ticker"] = match.get("ticker", "")
        else:
            row["kalshi_prob"] = np.nan
            row["kalshi_ticker"] = None

        # Polymarket
        poly_events = poly_fed if q["category"] == "fed_rate" else poly_gas
        match = match_polymarket_event(poly_events, q)
        if match:
            row["polymarket_prob"] = get_market_price(match, "polymarket")
        else:
            row["polymarket_prob"] = np.nan

        results.append(row)

    return pd.DataFrame(results)


market_probs = collect_market_probabilities(
    all_questions, kalshi_fed, kalshi_gas, poly_fed_events, poly_gas_events
)

print("Prediction Market Probabilities:")
print(market_probs[["question_id", "kalshi_prob", "polymarket_prob"]].to_string(index=False))

kalshi_coverage = market_probs["kalshi_prob"].notna().sum()
poly_coverage = market_probs["polymarket_prob"].notna().sum()
print(f"\nCoverage: Kalshi {kalshi_coverage}/{len(all_questions)}, Polymarket {poly_coverage}/{len(all_questions)}")

# Section 3: LLM Forecasting

## 3.1 Vanilla Prompting (No Tools)

Each model receives:
1. A **system prompt** establishing the forecasting persona
2. A **user prompt** with the specific binary question and resolution criteria
3. Instructions to output a probability between 0.0 and 1.0

No external data access â€” the model relies entirely on its training data and reasoning.

**Key design choice**: prompts deliberately exclude prediction market prices to ensure LLM forecasts are independent and can be fairly compared against market prices.

In [ ]:
# --- 3.1 Vanilla Prompting Setup ---

VANILLA_SYSTEM_PROMPT = """You are an expert forecaster and superforecaster. Your task is to estimate
the probability that a specific event will occur. You must provide a single
probability estimate between 0.0 (certainly will NOT happen) and 1.0 (certainly WILL happen).

Guidelines:
- Consider base rates and historical patterns
- Account for current economic conditions based on your training data
- Be well-calibrated: events you assign 70% probability should occur about 70% of the time
- Avoid anchoring to round numbers (0.5, 0.25, 0.75) unless truly justified
- Consider both arguments for and against the event occurring

You MUST end your response with exactly one line in this format:
PROBABILITY: X.XX

where X.XX is your probability estimate between 0.00 and 1.00."""

VANILLA_USER_TEMPLATE = """Today's date is {today_date}.

Question: {question_text}

Resolution criteria: {resolution_criteria}

Please reason through this step by step, then provide your probability estimate."""


def parse_probability_from_response(text):
    """Extract the probability value from an LLM response."""
    text = str(text)
    # Look for PROBABILITY: X.XX pattern
    match = re.search(r"PROBABILITY:\s*(0\.\d+|1\.00?|0\.0+|1\.0)", text)
    if match:
        return float(match.group(1))
    # Fallback: look for any decimal between 0 and 1 near the end
    matches = re.findall(r"\b(0\.\d+|1\.0)\b", text[-300:])
    if matches:
        return float(matches[-1])
    return np.nan


def get_llm_instance(model_key, temperature=0):
    """Factory function to create an LLM instance."""
    config = MODELS[model_key]
    if config["provider"] == "google":
        return ChatGoogleGenerativeAI(model=config["model_id"], temperature=temperature)
    elif config["provider"] == "openai":
        return ChatOpenAI(model=config["model_id"], temperature=temperature)
    elif config["provider"] == "anthropic":
        return ChatAnthropic(model=config["model_id"], temperature=temperature)
    else:
        raise ValueError(f"Unknown provider: {config['provider']}")


def get_resolution_criteria(question):
    """Build resolution criteria string for a question."""
    if question["category"] == "fed_rate":
        return (
            f"Resolves YES if the FRED federal funds target rate upper bound (DFEDTARU)"
            f" decreases after the {question['resolution_date']} FOMC meeting."
            f" Current target rate upper bound: {question['current_rate']:.2f}%."
        )
    else:
        return (
            f"Resolves YES if the EIA weekly US national average retail gasoline price"
            f" exceeds ${question['threshold']:.2f}/gal by {question['resolution_date']}."
            f" Current price: ${question['current_price']:.3f}/gal."
        )


print("Vanilla prompting setup complete.")
print(f"Models to evaluate: {', '.join(MODELS[k]['name'] for k in MODELS)}")

In [ ]:
# --- 3.1 Run Vanilla Forecasting ---

def run_vanilla_forecasting(questions, cache):
    """Run vanilla (no-tool) LLM forecasting across all models and questions."""
    results = []
    today = datetime.now().strftime("%Y-%m-%d")

    for model_key in MODELS:
        print(f"\nForecasting with {MODELS[model_key]['name']} (vanilla)...")
        llm = get_llm_instance(model_key, temperature=0)

        for q in tqdm(questions, desc=MODELS[model_key]["name"]):
            cache_key = get_cache_key("vanilla", model_key, q["id"])

            if cache_key in cache:
                output = cache[cache_key]["output"]
            else:
                criteria = get_resolution_criteria(q)
                messages = [
                    SystemMessage(content=VANILLA_SYSTEM_PROMPT),
                    HumanMessage(content=VANILLA_USER_TEMPLATE.format(
                        today_date=today,
                        question_text=q["text"],
                        resolution_criteria=criteria,
                    )),
                ]

                try:
                    response = llm.invoke(messages)
                    output = response.content
                except Exception as e:
                    print(f"  Error ({model_key}, {q['id']}): {e}")
                    output = f"ERROR: {str(e)}"

                cache[cache_key] = {"output": output}
                save_cache(cache)

            prob = parse_probability_from_response(output)
            results.append({
                "question_id": q["id"],
                "model": MODELS[model_key]["name"],
                "method": "vanilla",
                "probability": prob,
                "raw_output": str(output)[-300:],
            })

    return pd.DataFrame(results)


cache = load_cache()
vanilla_results = run_vanilla_forecasting(all_questions, cache)
print(f"\nCollected {len(vanilla_results)} vanilla forecasts")
print(vanilla_results.groupby("model")["probability"].describe().round(3))

## 3.2 Tool-Augmented LLM Forecasting

Now we give the same models access to real-time data tools:
1. **`get_federal_funds_rate`**: Fetch current and historical federal funds rate data from FRED
2. **`get_gas_prices`**: Fetch current and historical gasoline prices from EIA
3. **`get_fomc_schedule`**: Get the 2026 FOMC meeting schedule with past/upcoming status

We use LangChain's `bind_tools()` interface, which works across all three providers. The model decides which tools to call, receives the results, and then produces its forecast.

**The key question: does tool access improve forecasting accuracy?**

In [ ]:
# --- 3.2 Tool Definitions ---

@tool
def get_federal_funds_rate(lookback_days: int = 90) -> str:
    """Fetch the current and recent federal funds rate data from FRED.

    Args:
        lookback_days: Number of days of historical data to return (default 90)

    Returns:
        A string summary of the federal funds rate data.
    """
    fred = Fred(api_key=FRED_API_KEY)
    start = (datetime.now() - timedelta(days=lookback_days)).strftime("%Y-%m-%d")

    rate = fred.get_series("DFF", observation_start=start)
    target_upper = fred.get_series("DFEDTARU", observation_start=start)
    target_lower = fred.get_series("DFEDTARL", observation_start=start)

    current_rate = rate.dropna().iloc[-1]
    current_upper = target_upper.dropna().iloc[-1]
    current_lower = target_lower.dropna().iloc[-1]

    changes = target_upper.diff().dropna()
    cuts = changes[changes < 0]
    hikes = changes[changes > 0]

    return (
        f"Federal Funds Rate Data (last {lookback_days} days):\n"
        f"- Current effective rate: {current_rate:.2f}%\n"
        f"- Current target range: {current_lower:.2f}% - {current_upper:.2f}%\n"
        f"- Rate cuts in period: {len(cuts)} (total: {cuts.sum():.2f}pp)\n"
        f"- Rate hikes in period: {len(hikes)} (total: {hikes.sum():.2f}pp)\n"
        f"- Rate on {rate.dropna().index[-1].strftime('%Y-%m-%d')}: {current_rate:.2f}%\n"
        f"- Rate {lookback_days} days ago: {rate.dropna().iloc[0]:.2f}%"
    )


@tool
def get_gas_prices(weeks: int = 12) -> str:
    """Fetch recent US retail gasoline price data from the EIA.

    Args:
        weeks: Number of weeks of historical data to return (default 12)

    Returns:
        A string summary of gasoline price data.
    """
    params = {
        "api_key": EIA_API_KEY,
        "frequency": "weekly",
        "data[0]": "value",
        "facets[product][]": "EPMR",
        "facets[duoarea][]": "NUS",
        "sort[0][column]": "period",
        "sort[0][direction]": "desc",
        "offset": 0,
        "length": weeks,
    }
    response = requests.get(EIA_GAS_PRICE_URL, params=params)
    data = response.json()["response"]["data"]

    prices = [(d["period"], float(d["value"])) for d in data]
    prices.sort(key=lambda x: x[0])

    current = prices[-1][1]
    high = max(p[1] for p in prices)
    low = min(p[1] for p in prices)
    avg = sum(p[1] for p in prices) / len(prices)
    trend = current - prices[0][1]

    return (
        f"US Retail Gasoline Prices (last {weeks} weeks):\n"
        f"- Current price: ${current:.3f}/gal (week of {prices[-1][0]})\n"
        f"- {weeks}-week high: ${high:.3f}/gal\n"
        f"- {weeks}-week low: ${low:.3f}/gal\n"
        f"- {weeks}-week average: ${avg:.3f}/gal\n"
        f"- Trend: {'Up' if trend > 0 else 'Down'} ${abs(trend):.3f}/gal over {weeks} weeks\n"
        f"- Recent weekly prices: {', '.join(f'${p[1]:.3f}' for p in prices[-6:])}"
    )


@tool
def get_fomc_schedule() -> str:
    """Get the 2026 FOMC meeting schedule and status.

    Returns:
        A string listing upcoming FOMC meetings with dates and status.
    """
    lines = []
    for m in FOMC_DATES_2026:
        meeting_date = datetime.strptime(m["date"], "%Y-%m-%d")
        status = "PAST" if meeting_date < datetime.now() else "UPCOMING"
        lines.append(f"  {m['meeting']} 2026 ({m['date']}): {status}")

    return (
        "2026 FOMC Meeting Schedule:\n"
        + "\n".join(lines)
        + "\n\nNote: The Fed announces its rate decision on the second day of each meeting."
    )


TOOLS = [get_federal_funds_rate, get_gas_prices, get_fomc_schedule]
print(f"Defined {len(TOOLS)} tools: {[t.name for t in TOOLS]}")

In [ ]:
# --- 3.2 Run Tool-Augmented Forecasting ---

TOOL_SYSTEM_PROMPT = """You are an expert forecaster with access to real-time economic data tools.
Your task is to estimate the probability that a specific event will occur.

You have access to the following tools:
- get_federal_funds_rate: Fetch current and historical federal funds rate data
- get_gas_prices: Fetch recent US retail gasoline price data
- get_fomc_schedule: Get the 2026 FOMC meeting schedule

Instructions:
1. FIRST, use the relevant tools to gather current data
2. THEN, reason through the question using the data you retrieved
3. Consider base rates, trends, and current conditions
4. Provide a well-calibrated probability estimate

You MUST end your response with exactly one line in this format:
PROBABILITY: X.XX"""

TOOL_USER_TEMPLATE = """Today's date is {today_date}.

Question: {question_text}

Resolution criteria: {resolution_criteria}

Please use the available tools to gather relevant data, then reason through this
step by step and provide your probability estimate."""


def run_tool_augmented_forecasting(questions, cache):
    """Run tool-augmented LLM forecasting with LangChain tool calling."""
    results = []
    today = datetime.now().strftime("%Y-%m-%d")
    tool_map = {t.name: t for t in TOOLS}

    for model_key in MODELS:
        print(f"\nForecasting with {MODELS[model_key]['name']} (with tools)...")
        llm = get_llm_instance(model_key, temperature=0)
        llm_with_tools = llm.bind_tools(TOOLS)

        for q in tqdm(questions, desc=f"{MODELS[model_key]['name']} + tools"):
            cache_key = get_cache_key("tool", model_key, q["id"])

            if cache_key in cache:
                output = cache[cache_key]["output"]
            else:
                criteria = get_resolution_criteria(q)
                messages = [
                    SystemMessage(content=TOOL_SYSTEM_PROMPT),
                    HumanMessage(content=TOOL_USER_TEMPLATE.format(
                        today_date=today,
                        question_text=q["text"],
                        resolution_criteria=criteria,
                    )),
                ]

                # Agentic loop: let the model call tools until it produces a final answer
                output = "ERROR: max iterations reached"
                for _ in range(5):
                    try:
                        resp = llm_with_tools.invoke(messages)
                    except Exception as e:
                        output = f"ERROR: {str(e)}"
                        break

                    messages.append(resp)

                    if resp.tool_calls:
                        for tc in resp.tool_calls:
                            tool_fn = tool_map[tc["name"]]
                            tool_result = tool_fn.invoke(tc["args"])
                            messages.append(ToolMessage(
                                content=str(tool_result),
                                tool_call_id=tc["id"],
                            ))
                    else:
                        output = resp.content
                        break

                cache[cache_key] = {"output": output}
                save_cache(cache)

            prob = parse_probability_from_response(output)
            results.append({
                "question_id": q["id"],
                "model": MODELS[model_key]["name"],
                "method": "tool_augmented",
                "probability": prob,
                "raw_output": str(output)[-300:],
            })

    return pd.DataFrame(results)


cache = load_cache()
tool_results = run_tool_augmented_forecasting(all_questions, cache)
print(f"\nCollected {len(tool_results)} tool-augmented forecasts")
print(tool_results.groupby("model")["probability"].describe().round(3))

# Section 4: Scoring and Evaluation

## 4.1 Brier Score

$$BS = \frac{1}{N} \sum_{i=1}^{N} (p_i - o_i)^2$$

Reference benchmarks:
- **Perfect forecaster**: BS = 0.000
- **Always predict 50%** (no skill): BS = 0.250
- **Always 100% confident and wrong**: BS = 1.000

## 4.2 Hypothetical Returns

We simulate a threshold-based betting strategy:
- If forecast differs from market price by more than $\delta$ (default 10pp):
  - **Buy YES** if forecast > market + $\delta$ (cost = market price, payout = $1 if YES)
  - **Buy NO** if forecast < market - $\delta$ (cost = 1 - market price, payout = $1 if NO)
- Each bet is $1 notional

This tests whether the forecaster can identify **mispriced** markets.

In [ ]:
# --- 4.1 Resolution & Scoring ---

def resolve_questions(questions, fred_data, gas_prices):
    """Determine the actual outcomes for resolved questions."""
    outcomes = {}
    today = datetime.now()

    for q in questions:
        res_date = datetime.strptime(q["resolution_date"], "%Y-%m-%d")
        if res_date > today:
            outcomes[q["id"]] = np.nan  # Not yet resolved
            continue

        if q["category"] == "fed_rate":
            target = fred_data["target_upper"]
            pre = target[target.index < res_date]
            post = target[target.index >= res_date]
            if len(pre) > 0 and len(post) > 0:
                outcomes[q["id"]] = 1 if post.iloc[0] < pre.iloc[-1] else 0
            else:
                outcomes[q["id"]] = np.nan

        elif q["category"] == "gas_price":
            prices_before = gas_prices[
                gas_prices["period"] <= res_date.strftime("%Y-%m-%d")
            ]
            if len(prices_before) > 0:
                latest = prices_before["value"].iloc[-1]
                outcomes[q["id"]] = 1 if latest > q["threshold"] else 0
            else:
                outcomes[q["id"]] = np.nan

    return outcomes


def compute_brier_scores(forecast_df, outcomes):
    """Compute Brier scores for each forecast."""
    df = forecast_df.copy()
    df["outcome"] = df["question_id"].map(outcomes)
    df = df.dropna(subset=["outcome", "probability"])

    if len(df) == 0:
        print("WARNING: No resolved questions with valid forecasts. Brier scores cannot be computed.")
        return df

    df["brier_score"] = (df["probability"] - df["outcome"]) ** 2
    return df


def compute_returns(forecast_df, market_probs, outcomes, delta=0.10):
    """Compute hypothetical returns from threshold-based betting."""
    df = forecast_df.copy()
    df["outcome"] = df["question_id"].map(outcomes)

    # Use Kalshi as the market benchmark
    df = df.merge(
        market_probs[["question_id", "kalshi_prob"]],
        on="question_id",
        how="left",
    )
    df = df.dropna(subset=["outcome", "probability", "kalshi_prob"])

    rows = []
    for _, row in df.iterrows():
        p_forecast = row["probability"]
        p_market = row["kalshi_prob"]
        outcome = row["outcome"]

        if p_forecast > p_market + delta:
            profit = outcome * 1.0 - p_market
            rows.append({**row, "action": "BUY_YES", "profit": profit})
        elif p_forecast < p_market - delta:
            profit = (1 - outcome) * 1.0 - (1 - p_market)
            rows.append({**row, "action": "BUY_NO", "profit": profit})
        else:
            rows.append({**row, "action": "NO_BET", "profit": 0.0})

    return pd.DataFrame(rows)


print("Scoring functions defined.")

In [ ]:
# --- 4.2 Compute Results ---

# Combine all forecasts
all_forecasts = pd.concat([vanilla_results, tool_results], ignore_index=True)

# Add prediction market forecasts as their own "model"
market_rows = []
for _, row in market_probs.iterrows():
    if pd.notna(row.get("kalshi_prob")):
        market_rows.append({
            "question_id": row["question_id"],
            "model": "Kalshi (Market)",
            "method": "prediction_market",
            "probability": row["kalshi_prob"],
            "raw_output": "",
        })
    if pd.notna(row.get("polymarket_prob")):
        market_rows.append({
            "question_id": row["question_id"],
            "model": "Polymarket (Market)",
            "method": "prediction_market",
            "probability": row["polymarket_prob"],
            "raw_output": "",
        })

if market_rows:
    all_forecasts = pd.concat([all_forecasts, pd.DataFrame(market_rows)], ignore_index=True)

print(f"Total forecasts: {len(all_forecasts)}")
print(f"Forecasters: {all_forecasts.groupby(['model', 'method']).size().reset_index(name='n').to_string(index=False)}")

# Resolve questions
outcomes = resolve_questions(all_questions, fred_data, gas_prices)
resolved_count = sum(1 for v in outcomes.values() if pd.notna(v))
unresolved_count = sum(1 for v in outcomes.values() if pd.isna(v))
print(f"\nResolved: {resolved_count}/{len(outcomes)} questions")
print(f"Unresolved (future): {unresolved_count}/{len(outcomes)} questions")

for qid, outcome in outcomes.items():
    status = f"outcome={int(outcome)}" if pd.notna(outcome) else "pending"
    print(f"  {qid}: {status}")

# Compute Brier scores
scored_df = compute_brier_scores(all_forecasts, outcomes)
if len(scored_df) > 0:
    print("\n" + "=" * 60)
    print("BRIER SCORES")
    print("=" * 60)
    brier_summary = scored_df.groupby(["method", "model"])["brier_score"].agg(["mean", "std", "count"])
    brier_summary.columns = ["Mean Brier", "Std Brier", "N"]
    print(brier_summary.sort_values("Mean Brier").round(4))
else:
    print("\nNo questions have resolved yet -- Brier scores will be available after resolution dates pass.")

# Compute returns
returns_df = compute_returns(all_forecasts, market_probs, outcomes, delta=0.10)
if len(returns_df) > 0 and returns_df["profit"].abs().sum() > 0:
    print("\n" + "=" * 60)
    print("HYPOTHETICAL RETURNS (delta=0.10)")
    print("=" * 60)
    returns_summary = returns_df.groupby(["method", "model"])["profit"].agg(["sum", "mean", "count"])
    returns_summary.columns = ["Total P&L ($)", "Avg P&L/Bet ($)", "N Bets"]
    print(returns_summary.sort_values("Total P&L ($)", ascending=False).round(4))

# Section 5: Visualizations

In [ ]:
# --- 5.1 Brier Score and Returns Charts ---

fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Plot 1: Mean Brier Score by forecaster
if len(scored_df) > 0:
    brier_by_method = scored_df.groupby(["model", "method"])["brier_score"].mean().reset_index()
    sns.barplot(data=brier_by_method, x="model", y="brier_score", hue="method", ax=axs[0])
    axs[0].set_title("Mean Brier Score by Forecaster\n(Lower is Better)")
    axs[0].set_xlabel("")
    axs[0].set_ylabel("Mean Brier Score")
    axs[0].tick_params(axis="x", rotation=45)
    axs[0].axhline(y=0.25, color="red", linestyle="--", alpha=0.5, label="No-skill (0.25)")
    axs[0].legend(fontsize=7)
else:
    axs[0].text(0.5, 0.5, "No resolved questions yet", ha="center", va="center", transform=axs[0].transAxes)
    axs[0].set_title("Mean Brier Score (pending)")

# Plot 2: Brier Score by question category
if len(scored_df) > 0:
    q_categories = pd.DataFrame(all_questions)[["id", "category"]].rename(columns={"id": "question_id"})
    scored_with_cat = scored_df.merge(q_categories, on="question_id")
    sns.barplot(data=scored_with_cat, x="category", y="brier_score", hue="method", ax=axs[1])
    axs[1].set_title("Brier Score by Category")
    axs[1].set_xlabel("")
    axs[1].set_ylabel("Mean Brier Score")
    axs[1].legend(fontsize=7)
else:
    axs[1].text(0.5, 0.5, "No resolved questions yet", ha="center", va="center", transform=axs[1].transAxes)
    axs[1].set_title("Brier by Category (pending)")

# Plot 3: Cumulative returns
if len(returns_df) > 0 and returns_df["profit"].abs().sum() > 0:
    for name, group in returns_df.groupby(["model", "method"]):
        label = f"{name[0]} ({name[1]})"
        cumulative = group["profit"].cumsum()
        axs[2].plot(range(len(cumulative)), cumulative, label=label, marker="o", markersize=3)
    axs[2].set_title("Cumulative Hypothetical Returns")
    axs[2].set_xlabel("Bet Number")
    axs[2].set_ylabel("Cumulative P&L ($)")
    axs[2].axhline(y=0, color="black", linestyle="-", linewidth=0.5)
    axs[2].legend(fontsize=6)
else:
    axs[2].text(0.5, 0.5, "No bets placed yet", ha="center", va="center", transform=axs[2].transAxes)
    axs[2].set_title("Cumulative Returns (pending)")

plt.tight_layout()
plt.show()

In [ ]:
# --- 5.2 Calibration Plot ---

def plot_calibration(scored_df, n_bins=5):
    """Plot calibration curves for each forecaster."""
    fig, ax = plt.subplots(figsize=(8, 8))

    for (model, method), group in scored_df.groupby(["model", "method"]):
        probs = group["probability"].values
        outcomes_arr = group["outcome"].values

        bins = np.linspace(0, 1, n_bins + 1)
        bin_means = []
        bin_freqs = []
        for i in range(n_bins):
            mask = (probs >= bins[i]) & (probs < bins[i + 1])
            if mask.sum() > 0:
                bin_means.append(probs[mask].mean())
                bin_freqs.append(outcomes_arr[mask].mean())

        if bin_means:
            ax.plot(bin_means, bin_freqs, marker="o", label=f"{model} ({method})")

    ax.plot([0, 1], [0, 1], "k--", label="Perfect calibration")
    ax.set_xlabel("Predicted Probability")
    ax.set_ylabel("Observed Frequency")
    ax.set_title("Calibration Plot")
    ax.legend(loc="lower right", fontsize=7)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    plt.tight_layout()
    plt.show()


if len(scored_df) > 0:
    plot_calibration(scored_df)
else:
    print("Calibration plot will be available once questions resolve.")

In [ ]:
# --- 5.3 Forecast Comparison Heatmap ---

def plot_forecast_heatmap(forecasts, questions):
    """Heatmap of all probabilities: questions (rows) x forecasters (columns)."""
    forecasts = forecasts.copy()
    forecasts["forecaster"] = forecasts["model"] + "\n(" + forecasts["method"] + ")"

    pivot = forecasts.pivot_table(
        index="question_id",
        columns="forecaster",
        values="probability",
    )

    fig, ax = plt.subplots(figsize=(16, max(8, len(pivot) * 0.5)))
    sns.heatmap(
        pivot, annot=True, fmt=".2f", cmap="RdYlGn", center=0.5,
        vmin=0, vmax=1, ax=ax, cbar_kws={"label": "Probability"},
    )
    ax.set_title("Forecast Comparison Heatmap")
    ax.set_ylabel("Question")
    ax.set_xlabel("")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()


plot_forecast_heatmap(all_forecasts, all_questions)

# Section 6: Discussion and Conclusions

## Key Questions

1. **Do prediction markets outperform LLMs?**
   - Compare Brier scores of Kalshi/Polymarket vs. vanilla LLMs vs. tool-augmented LLMs
   - Markets aggregate information from many participants; can a single LLM match this?

2. **Does tool access improve LLM forecasting?**
   - Compare vanilla vs. tool-augmented Brier scores for each model
   - Real-time data should help, but does the model use it effectively?

3. **Which LLM is the best forecaster?**
   - Rank GPT-5, Gemini, Claude by Brier score and returns
   - Does the ranking change between vanilla and tool-augmented conditions?

4. **Are there category-specific patterns?**
   - Fed rate questions may favor models with strong economic reasoning
   - Gas price questions may favor models with access to trend data

5. **Can any forecaster generate positive returns against the market?**
   - A positive total P&L means the forecaster identified genuine mispricings
   - How sensitive are returns to the betting threshold delta?

## Limitations

- **Small sample size**: limited by the number of resolvable questions within the project timeframe
- **Single market snapshot**: prediction market prices were captured at one point in time (markets update continuously)
- **LLM training cutoffs**: models may lack recent economic data in their training, which is exactly what tool augmentation addresses
- **Question design**: our questions may not perfectly overlap with existing prediction market contracts
- **Not all questions may resolve**: FOMC meetings later in 2026 won't have outcomes during the semester

In [ ]:
# --- 6.1 Summary Statistics ---

if len(scored_df) > 0:
    summary = scored_df.groupby(["method", "model"]).agg({
        "brier_score": ["mean", "std"],
        "probability": ["mean", "std"],
        "question_id": "count",
    }).round(4)
    summary.columns = ["Mean Brier", "Std Brier", "Mean Prob", "Std Prob", "N"]
    summary = summary.sort_values("Mean Brier")

    print("=" * 70)
    print("FINAL RESULTS: Forecaster Ranking by Brier Score")
    print("=" * 70)
    print(summary)
    print(f"\nBaseline (always predict 0.5): Brier = 0.2500")
    print(f"Perfect forecaster: Brier = 0.0000")
else:
    print("=" * 70)
    print("FORECAST SUMMARY (questions not yet resolved)")
    print("=" * 70)
    prob_summary = all_forecasts.groupby(["method", "model"])["probability"].agg(["mean", "std", "count"])
    prob_summary.columns = ["Mean Prob", "Std Prob", "N"]
    print(prob_summary.round(4))
    print("\nBrier scores and returns will be computed after questions resolve.")
    print("Re-run this notebook after resolution dates to see final results.")